### Interactivity

In [1]:
import altair as alt
import pandas as pd

cocktails = pd.read_csv('../data/cocktails.tsv', delimiter='\t')

In [2]:
cocktails.head(2)

,name,abv,acid,sugar,type,index,instructions,ingredients,ncotw
0,Pisco Sour,12.1,0.68,7.2,eggwhite,5,"Dry shake, shake with ice, coupe. Add 3 drops ...",2 oz pisco (40% abv)<br/>1 oz egg white<br/>3/...,not yet
1,Pink Lady,12.4,0.64,9.0,eggwhite,6,"Dry shake, shake with ice, coupe.",1 1/2 oz Plymouth gin<br/>1 oz egg white<br/>3...,not yet


In [3]:
alt.Chart(cocktails).mark_circle().encode(x='acid', y='sugar')

alt.Chart(...)

In [4]:
alt.Chart(cocktails).mark_circle().encode(x='acid', y='sugar').interactive()

alt.Chart(...)

Not super interesting, though...

In [5]:
three = cocktails[cocktails['type'].isin(['shaken', 'stirred', 'blended'])]

In [6]:
# common config
brush = alt.selection(type='interval')
base = alt.Chart(three).add_selection(brush)

# points
points = base.mark_circle(size=100).encode(
    x='acid',
    y='sugar',
    color=alt.condition(brush, 'type', alt.value('grey'))
)

points

alt.Chart(...)

In [7]:
base.mark_tick().encode(
    x='acid',
    y='type',
    color=alt.condition(brush, 'type', alt.value('lightgrey'))
)

alt.Chart(...)

In [8]:
base.mark_tick().encode(
    alt.X('sugar', axis=alt.Axis(labels=False, domain=False, ticks=False)),
    alt.Y('type', title='', axis=alt.Axis(labels=False, domain=False, ticks=False)),
    color=alt.condition(brush, 'type', alt.value('lightgrey'))
)

alt.Chart(...)

In [10]:
# common ticks
tick_axis = alt.Axis(labels=False, domain=False, ticks=False)

sugar = base.mark_tick().encode(
    alt.X('sugar', axis=tick_axis),
    alt.Y('type', title='', axis=tick_axis),
    color=alt.condition(brush, 'type', alt.value('lightgrey'))
)

acid = base.mark_tick().encode(
    alt.X('acid', title='', axis=tick_axis),
    alt.Y('type', axis=tick_axis),
    color=alt.condition(brush, 'type', alt.value('lightgrey'))
)

sugar & acid

alt.VConcatChart(...)

In [15]:
# common config
brush = alt.selection(type='interval')
base = alt.Chart(three).add_selection(brush)

# points
points = base.mark_circle(size=100).encode(
    x='acid',
    y='sugar',
    color=alt.condition(brush, 'type', alt.value('grey'))
)

# ticks
tick_axis = alt.Axis(labels=False, domain=False, ticks=False)

sugar = base.mark_tick().encode(
    alt.X('sugar', axis=tick_axis),
    alt.Y('type', title='', axis=tick_axis),
    color=alt.condition(brush, 'type', alt.value('lightgrey'))
)

acid = base.mark_tick().encode(
    alt.X('type', title='', axis=tick_axis),
    alt.Y('acid', axis=tick_axis),
    color=alt.condition(brush, 'type', alt.value('lightgrey'))
)

# all together
acid | (points & sugar)

alt.HConcatChart(...)

In [ ]:
alt.Chart(marathon).mark_line().encode(
    x='time',
    y='distance',
    color='runner'
).interactive()

In [ ]:
import altair as alt
from vega_datasets import data

source = data.seattle_weather()
brush = alt.selection(type='interval', encodings=['x'])

bars = alt.Chart().mark_bar().encode(
    x='month(date):O',
    y='mean(precipitation):Q',
    opacity=alt.condition(brush, alt.OpacityValue(1), alt.OpacityValue(0.7)),
).add_selection(
    brush
)

line = alt.Chart().mark_rule(color='firebrick').encode(
    y='mean(precipitation):Q',
    size=alt.SizeValue(3)
).transform_filter(
    brush
)

alt.layer(bars, line, data=source)


In [16]:
# https://github.com/altair-viz/altair/issues/984

import altair as alt
from vega_datasets import data

stockdata = data.stocks()
stockdata['pe'] = stockdata['price'] / 10

In [17]:
selector = alt.selection_single(
    fields=['symbol'], 
    empty='all',
    init={'symbol': 'AAPL'},
    bind='legend'
)

price = alt.Chart(stockdata).mark_line(point=True).encode(
    x='date:T',
    y='price:Q',
    color='symbol:N',
    opacity=alt.condition(selector, alt.value(1), alt.value(0))
).add_selection(
    selector
)

price

alt.Chart(...)